# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: define 2 labels for good or bad review
* Model: distilbert-base-uncased-finetuned-sst-2-english
* Evaluation approach: I used the trainer along with a compute_metrics function to calculate the mean for the labels equaling the predictions 
* Fine-tuning dataset: imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [67]:
from transformers import DistilBertTokenizer, DistilBertModel
from datasets import load_dataset

import torch 

dataset = load_dataset("imdb")
test_sentence = dataset['test'][0]['text']
print(test_sentence)

I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they have to alway

In [68]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

device = "cuda"
#tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model.to(device)

inputs = tokenizer(test_sentence, return_tensors="pt")
inputs.to(device)

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]


'NEGATIVE'

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [69]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english",
    num_labels=2,
    id2label={0: "NOT GOOD REVIEW", 1: "GOOD REVIEW"},
    label2id={"NOT GOOD REVIEW": 0, "GOOD REVIEW": 1})
model.to(device)


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [70]:
for param in model.parameters():
    param.requires_grad=True

print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [72]:
splits = ["train", "test"]
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True)

tokenized_dataset["train"]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [73]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir ="./data/good_bad",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.204100,0.186880,0.931000


Checkpoint destination directory ./data/good_bad/checkpoint-1563 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1563, training_loss=0.2292033504844856, metrics={'train_runtime': 1638.8159, 'train_samples_per_second': 15.255, 'train_steps_per_second': 0.954, 'total_flos': 3280166004732288.0, 'train_loss': 0.2292033504844856, 'epoch': 1.0})

In [74]:
tokenized_dataset["test"]["text"][15000]

'To think this film was made the year I was born. To think people are still having their constitutional rights taken away, now in the name of "homeland security". To think this movie was intentionally banned from the American public. PUNISHMENT PARK addresses the political divide in the United States better than any movie I\'ve ever seen. Had it been more widely seen, would it have changed anything? A movie like this is so polarizing, it has the potential to cause riots. It shakes you up and forces you to take sides. It makes you face the issue: are you for the people\'s right of dissent in a time of war, or for the constitution being compromised in the name of "national security"? The protagonists are forced by the government to race to the American flag in a game that undermines the very ideals the flag stands for. The acting is totally convincing. So much so, I can\'t see any acting going on here at all. If this is a scripted documentary, it\'s more convincing than any reality show 

In [75]:
tokenized_dataset["test"]["label"][15000]

1

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [76]:
import pandas as pd

items = tokenized_dataset["test"].select([0,3,40, 15000, 20000])
results = trainer.predict(items)
print(results)
df = pd.DataFrame(
    {
        "review": [item["text"] for item in items],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids
    }
)

pd.set_option("display.max_colwidth", None)
df

PredictionOutput(predictions=array([[ 3.3489106, -2.79801  ],
       [ 2.5921514, -2.2020483],
       [ 3.6296167, -3.0127168],
       [-1.4923091,  1.554837 ],
       [-2.3062415,  2.422587 ]], dtype=float32), label_ids=array([0, 0, 0, 1, 1]), metrics={'test_loss': 0.013376699760556221, 'test_accuracy': 1.0, 'test_runtime': 0.1117, 'test_samples_per_second': 44.773, 'test_steps_per_second': 8.955})


,review,predictions,labels
0,"I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they have to always say ""Gene Roddenberry's Earth..."" otherwise people would not continue watching. Roddenberry's ashes must be turning in their orbit as this dull, cheap, poorly edited (watching it without advert breaks really brings this home) trudging Trabant of a show lumbers into space. Spoiler. So, kill off a main character. And then bring him back as another actor. Jeeez! Dallas all over again.",0,0
1,"STAR RATING: ***** Saturday Night **** Friday Night *** Friday Morning ** Sunday Night * Monday Morning <br /><br />Former New Orleans homicide cop Jack Robideaux (Jean Claude Van Damme) is re-assigned to Columbus, a small but violent town in Mexico to help the police there with their efforts to stop a major heroin smuggling operation into their town. The culprits turn out to be ex-military, lead by former commander Benjamin Meyers (Stephen Lord, otherwise known as Jase from East Enders) who is using a special method he learned in Afghanistan to fight off his opponents. But Jack has a more personal reason for taking him down, that draws the two men into an explosive final showdown where only one will walk away alive.<br /><br />After Until Death, Van Damme appeared to be on a high, showing he could make the best straight to video films in the action market. While that was a far more drama oriented film, with The Shepherd he has returned to the high-kicking, no brainer action that first made him famous and has sadly produced his worst film since Derailed. It's nowhere near as bad as that film, but what I said still stands.<br /><br />A dull, predictable film, with very little in the way of any exciting action. What little there is mainly consists of some limp fight scenes, trying to look cool and trendy with some cheap slo-mo/sped up effects added to them that sadly instead make them look more desperate. Being a Mexican set film, director Isaac Florentine has tried to give the film a Robert Rodriguez/Desperado sort of feel, but this only adds to the desperation.<br /><br />VD gives a particularly uninspired performance and given he's never been a Robert De Niro sort of actor, that can't be good. As the villain, Lord shouldn't expect to leave the beeb anytime soon. He gets little dialogue at the beginning as he struggles to muster an American accent but gets mysteriously better towards the end. All the supporting cast are equally bland, and do nothing to raise the films spirits at all.<br /><br />This is one shepherd that's strayed right from the flock. *",0,0
2,"I can't believe the high marks people have given this film on this site. The writing is incredibly bad with people coming in at just the right time and revealing exactly what the heroine is doing to try to escape. (Don't you just hate it when that happens?). And the acting is so very, very bad that you may get a splinter in your eye from all the scenery being chewed.<br /><br />A nut is holding her hostage, children are outside the open screen, so she whimpers for help i

In [77]:
review = tokenized_dataset["test"]["text"][15000]
inputs = tokenizer(review, return_tensors="pt")
inputs.to(device)

#items = tokenized_dataset["test"].select([0,3,40, 15000, 20000])

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

# # Get the predicted class (corresponding to the highest logit)
# predictions = torch.argmax(outputs.logits, dim=-1)

'GOOD REVIEW'